In [8]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [9]:
from datasets import load_dataset

data_files = {"train": "/Users/heyodogo/Downloads/emobank_train.csv", "test": "/Users/heyodogo/Downloads/emobank_test.csv"}

dataset = load_dataset('csv', data_files = data_files)

dataset = dataset.remove_columns(['id', 'A', 'D', 'split'])

dataset = dataset.rename_column("V", "labels")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")


tokenized_datasets = dataset.map(tokenize_function, batched=True)

# These steps are necessary to convert the dataset into something that pytorch expects
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

tokenized_datasets.set_format("torch")

In [10]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader

SAMPLE_SET_SIZE = 300

# creating a small, 300 example dataset to train and evaluate our model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(SAMPLE_SET_SIZE*3))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(SAMPLE_SET_SIZE))

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    small_train_dataset, shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    small_eval_dataset, batch_size=8, collate_fn=data_collator
)

In [11]:
# from datasets import load_dataset
# from transformers import AutoTokenizer, DataCollatorWithPadding
# from torch.utils.data import DataLoader

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     tokenized_datasets["test"], batch_size=8, collate_fn=data_collator


In [12]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 512]),
 'token_type_ids': torch.Size([8, 512]),
 'attention_mask': torch.Size([8, 512])}

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(5.4784, grad_fn=<MseLossBackward0>) torch.Size([8, 1])


In [15]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
# print(num_training_steps)

/Users/heyodogo/Documents/DrOng/jupyter/class-env/lib/python3.11/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [16]:
import torch
import torch.nn as nn

loss_fn = nn.MSELoss()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [17]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)

        # Replace with your custom loss calculation
        predictions = outputs.logits  # Assuming 'logits' is the regression output
        targets = batch['labels']# Assuming 'labels' is the ground truth
        targets = targets.float()
        targets = targets.view(-1, 1)  # Reshape targets to match predictions
        loss = loss_fn(predictions, targets)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        # loss.backward()
        # optimizer.step()
        # lr_scheduler.step()
        # optimizer.zero_grad()
        # progress_bar.update(1)

  0%|          | 0/339 [00:00<?, ?it/s]

In [18]:
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.0}